In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Activation
from keras.models import Sequential
from keras.callbacks import TensorBoard
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
import h5py
import numpy as np
import pickle
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import models
from keras.utils import np_utils
%matplotlib inline

Using TensorFlow backend.


In [2]:
batch_size = 50
nb_classes = 10
nb_epoch = 100
data_augmentation = False

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('X_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
print('y_train shape:', y_train.shape)

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples
y_train shape: (50000, 10)


In [4]:
#Data normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [5]:
#loop to run all the models 
model=models.build_model_2()
model.compile(optimizer=SGD(momentum=0.9, lr=0.001), metrics=['accuracy'], loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        4864      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 1, 64)          102464    
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
__________

In [6]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.3,
              shuffle=True,
              callbacks=[TensorBoard(log_dir='./log', histogram_freq=4,write_graph=True,write_images=True,embeddings_freq=4)])

else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    #Only required if featurewise_center or featurewise_std_normalization or zca_whitening
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(x_train, y_train,
                        batch_size=batch_size),
                        samples_per_epoch=x_train.shape[0], #it should be equal to x_train.shape[0]/batch_size
                        epochs=nb_epoch,
                        callbacks=[TensorBoard(log_dir='./logGen', histogram_freq=4,write_graph=True,write_images=True,embeddings_freq=4)])

model.save('model2.h5')


Not using data augmentation.
Train on 35000 samples, validate on 15000 samples
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_1/bias:0 is illegal; using conv2d_1/bias_0 instead.
INFO:tensorflow:Summary name conv2d_1/bias:0 is illegal; using conv2d_1/bias_0 instead.
INFO:tensorflow:Summary name conv2d_2/kernel:0 is illegal; using conv2d_2/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_2/kernel:0 is illegal; using conv2d_2/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_2/bias:0 is illegal; using conv2d_2/bias_0 instead.
INFO:tensorflow:Summary name conv2d_2/bias:0 is illegal; using conv2d_2/bias_0 instead.
INFO:tensorflow:Summary name conv2d_3/kernel:0 is illegal; using conv2d_3/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_3/kernel:0 is illegal; using conv2d_3/kernel_0 instead.
INFO:tensorflow:S

InternalError: Dst tensor is not initialized.
	 [[Node: _recv_conv2d_1_input_0/_89 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_123__recv_conv2d_1_input_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

In [ ]:
values = model.predict(x_train)
values.shape()

In [ ]:
value_labels = model.predict_classes(x_train)